In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
stocks_df= pd.read_csv("/kaggle/input/india-stock-market/stocks_df.csv")
nse_indexes_df= pd.read_csv("/kaggle/input/india-stock-market/nse_indexes.csv")
indexes_df= pd.read_csv("/kaggle/input/india-stock-market/indexes_df.csv")

In [ ]:
stocks_df

In [ ]:
nse_indexes_df

In [ ]:
indexes_df.head()

In [ ]:
#doing for nifty_50

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# import yfinance as yf
try:
    import pandas_ta as ta
except:
    !pip install pandas_ta
    import pandas_ta as ta
# data = yf.download(tickers = '^RUI', start = '2012-03-11',end = '2022-07-10')
# data.head(10)




In [ ]:
data_nifty_50_orig = nse_indexes_df[nse_indexes_df.Index=="NIFTY 50"]
data_nifty_50 = data_nifty_50_orig.copy()
data_nifty_50

In [ ]:
data_nifty_50.describe()

In [ ]:
data_nifty_50.info()

In [ ]:
# Adding indicators
data_nifty_50['RSI']=ta.rsi(data_nifty_50.Close, length=15)
data_nifty_50['EMAF']=ta.ema(data_nifty_50.Close, length=20)
data_nifty_50['EMAM']=ta.ema(data_nifty_50.Close, length=100)
data_nifty_50['EMAS']=ta.ema(data_nifty_50.Close, length=150)


data_nifty_50['TargetNextClose'] = data_nifty_50['Close'].shift(-1)


In [ ]:
data_nifty_50

In [ ]:
data_nifty_50.dropna(inplace=True)
data_nifty_50

In [ ]:
data_nifty_50.drop(['Date', 'Index', 'Currency'], axis=1, inplace=True)
data_nifty_50

In [ ]:
data_nifty_50.reset_index(inplace = True)
data_nifty_50

In [ ]:
data_nifty_50.drop(['index'], axis=1, inplace = True)
data_nifty_50

In [ ]:
#maybe also remove volume
data_nifty_50.drop(['Volume'], axis=1, inplace = True)
data_nifty_50

# Model 1: Predict TargetNextClose(Next Day Closing Price)
* Open
* High
* Low
* Close
* RSI
* EMAF
* EMAM
* EMAS

In [ ]:
data_set_1= data_nifty_50.copy()
data_set_1

In [ ]:
import matplotlib.pyplot as plt
plt.plot(data_set_1.Close)

#Date vs Close price

In [ ]:
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range=(0,1))
data_set_scaled_1 = sc.fit_transform(data_set_1)
print(data_set_scaled_1)

In [ ]:
data_set_scaled_1.shape

In [ ]:
data_set_scaled_1[0:, 8].shape

In [ ]:
# multiple feature from data provided to the model
X1 = []
#print(data_set_scaled[0].size)
#data_set_scaled=data_set.values
backcandles = 30
print(data_set_scaled_1.shape[0])

for j in range(8):#data_set_scaled[0].size):#2 columns are target not X
    X1.append([])
    for i in range(backcandles, data_set_scaled_1.shape[0]):#backcandles+2
        X1[j].append(data_set_scaled_1[i-backcandles:i, j])

#move axis from 0 to position 2
X1=np.moveaxis(X1, [0], [2])

#Erase first elements of y because of backcandles to match X length
#del(yi[0:backcandles])
#X, yi = np.array(X), np.array(yi)
# Choose -1 for last column, classification else -2...
X1, yi1 =np.array(X1), np.array(data_set_scaled_1[backcandles:,-1])
y1=np.reshape(yi1,(len(yi1),1))
#y=sc.fit_transform(yi)
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X1)
print(X1.shape)
print(y1)
print(y1.shape)

In [ ]:
# split data into train test sets
splitlimit = int(len(X1)*0.8)
print(splitlimit)
X_train1, X_test1 = X1[:splitlimit], X1[splitlimit:]
y_train1, y_test1 = y1[:splitlimit], y1[splitlimit:]
print(X_train1.shape)
print(X_test1.shape)
print(y_train1.shape)
print(y_test1.shape)
print(y_train1)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import Dense
from keras.layers import TimeDistributed

import tensorflow as tf
import keras
from keras import optimizers
from keras.callbacks import History
from keras.models import Model
from keras.layers import Dense, Dropout, LSTM, Input, Activation, concatenate
import numpy as np
#tf.random.set_seed(20)
np.random.seed(10)

lstm_input = Input(shape=(backcandles, 8), name='lstm_input')
inputs = LSTM(150, name='first_layer')(lstm_input)
inputs = Dense(1, name='dense_layer')(inputs)
output = Activation('linear', name='output')(inputs)
model = Model(inputs=lstm_input, outputs=output)
adam = optimizers.Adam()
model.compile(optimizer=adam, loss='mse')
model.fit(x=X_train1, y=y_train1, batch_size=15, epochs=30, shuffle=True, validation_split = 0.1)